# D605 Task 3 - Identification of the Objective Function and Constraints

## A. Optimization Problem Program

> Using the mathematical representations that you developed in Task 2, write a program in either Python or R to solve the optimization problem computationally.

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 62.8 MB/s eta 0:00:00


In [2]:
import pulp
import sys

In [3]:
# Data definitions as described from "Amazon Distribution.docx"
# --- Define the sets of locations ---
hubs = ['CVG', 'AFW']
focus_cities = ['Leipzig', 'Hyderabad', 'San Bernardino']
fulfillment_centers = [
    'Paris', 'Cologne', 'Hanover', 'Bangalore',
    'Coimbatore', 'Delhi', 'Mumbai', 'Cagliari',
    'Catania', 'Milan', 'Rome', 'Katowice',
    'Barcelona', 'Madrid', 'Castle Donington', 'London',
    'Mobile', 'Anchorage', 'Fairbanks', 'Phoenix',
    'Los Angeles', 'Ontario', 'Riverside', 'Sacramento',
    'San Francisco', 'Stockton', 'Denver', 'Hartford',
    'Miami', 'Lakeland', 'Tampa', 'Atlanta',
    'Honolulu', 'Kahului/Maui', 'Kona', 'Chicago',
    'Rockford', 'Fort Wayne', 'South Bend', 'Des Moines',
    'Wichita', 'New Orleans', 'Baltimore', 'Minneapolis',
    'Kansas City', 'St. Louis', 'Omaha', 'Manchester',
    'Albuquerque', 'New York', 'Charlotte', 'Toledo',
    'Wilmington', 'Portland', 'Allentown', 'Pittsburgh',
    'San Juan', 'Nashville', 'Austin', 'Dallas',
    'Houston', 'San Antonio', 'Richmond', 'Seattle/Tacoma',
    'Spokane'
]

# --- Define capacities for hubs and focus cities ---
hub_capacities_per_month = {
    'CVG': 95_650,
    'AFW': 44_350
}

focus_city_capacities_per_month = {
    'Leipzig': 85_000,
    'Hyderabad': 19_000,
    'San Bernardino': 36_000
}

# --- Define demand for each fulfillment center ---
center_demands_per_month = {
    'Paris': 6500, 'Cologne': 640, 'Hanover': 180, 'Bangalore': 9100,
    'Coimbatore': 570, 'Delhi': 19000, 'Mumbai': 14800, 'Cagliari': 90,
    'Catania': 185, 'Milan': 800, 'Rome': 1700, 'Katowice': 170,
    'Barcelona': 2800, 'Madrid': 3700, 'Castle Donington': 30, 'London': 6700,
    'Mobile': 190, 'Anchorage': 175, 'Fairbanks': 38, 'Phoenix': 2400,
    'Los Angeles': 7200, 'Ontario': 100, 'Riverside': 1200, 'Sacramento': 1100,
    'San Francisco': 1900, 'Stockton': 240, 'Denver': 1500, 'Hartford': 540,
    'Miami': 3400, 'Lakeland': 185, 'Tampa': 1600, 'Atlanta': 3000,
    'Honolulu': 500, 'Kahului/Maui': 16, 'Kona': 63, 'Chicago': 5100,
    'Rockford': 172, 'Fort Wayne': 200, 'South Bend': 173, 'Des Moines': 300,
    'Wichita': 290, 'New Orleans': 550, 'Baltimore': 1300, 'Minneapolis': 1700,
    'Kansas City': 975, 'St. Louis': 1200, 'Omaha': 480, 'Manchester': 100,
    'Albuquerque': 450, 'New York': 11200, 'Charlotte': 900, 'Toledo': 290,
    'Wilmington': 150, 'Portland': 1200, 'Allentown': 420, 'Pittsburgh': 1000,
    'San Juan': 1100, 'Nashville': 650, 'Austin': 975, 'Dallas': 3300,
    'Houston': 3300, 'San Antonio': 1100, 'Richmond': 600, 'Seattle/Tacoma': 2000,
    'Spokane': 260
}

### A1. Demonstrate Solver Solution

> Demonstrate that the solver provided a solution to the optimization problem.

## B. Output Analysis

> Analyze the output of your model to demonstrate that the solution satisfies the requirements of the problem.

### B1. Satisfying Optimization Problem Constraints

> Demonstrate that the constraints of the optimization problem are satisfied.

### B2. Decision Variables, Constraints, and Objective Function

> Demonstrate that the solution includes decision variables, constraints, and the objective function.

### B3. Expected Output Solution

> Explain why the solution that your code outputs matches your expected output.

## C. Reflection

> Provide a reflection on how the development of your approach matched your expectations of the process.

## D. Sources

> Acknowledge sources, using in-text citations and references, for content that is quoted, paraphrased, or summarized.



* WGU Course Materials